In [6]:
#Project 2 - Hidden Markov Model
#Creator - Siddharth Pani and Abhishek Sharma
#Subject - Pattern Recognition 

In [7]:
import numpy as np
import pandas as pd

def loadData():
    test_set=pd.read_csv("Project2Data.txt",names=["input", "output"])
    test_set.head()
    return test_set

def CreateTransitionProbs(df):
    state_seq=df['input']
    state_occurrence_dic = {}
    for i in range(1,len(state_seq)):
        if state_seq[i-1] not in state_occurrence_dic:
            state_occurrence_dic[state_seq[i-1]] = {state_seq[i]:1}
        else:
            if state_seq[i] not in state_occurrence_dic[state_seq[i-1]]:
                state_occurrence_dic[state_seq[i-1]][state_seq[i]] = 1
            else:
                state_occurrence_dic[state_seq[i-1]][state_seq[i]] += 1
    #sunny = np.array(list(state_occurrence_dic['sunny'].values()))/float(sum(state_occurrence_dic['sunny'].values()))
    #rainy = np.array(list(state_occurrence_dic['rainy'].values()))/float(sum(state_occurrence_dic['rainy'].values()))
    #foggy = np.array(list(state_occurrence_dic['foggy'].values()))/float(sum(state_occurrence_dic['foggy'].values()))
    print(state_occurrence_dic)
    dict_df=pd.DataFrame.from_dict(state_occurrence_dic)
    print(dict_df)
    a_df=pd.DataFrame()
    a_df['foggy']=dict_df['foggy']/sum(state_occurrence_dic['foggy'].values())
    a_df['rainy']=dict_df['rainy']/sum(state_occurrence_dic['rainy'].values())
    a_df['sunny']=dict_df['sunny']/sum(state_occurrence_dic['sunny'].values())
    print('Transistion Probabilities')
    return a_df.transpose().round(2)

def CreateEmitionProbs(df):
    yes=[]
    no=[]
    s=[]
    for input in list(df['input'].unique()):
        for output in list(df['output'].unique()):
            if(output=='yes'):
                yes.append(len(df.loc[(df['input'] == input) & (df['output'] == 'yes')])/len(df.loc[(df['input'] == input)]))
            if(output=='no'):
                no.append(len(df.loc[(df['input'] == input) & (df['output'] == 'no')])/len(df.loc[(df['input'] == input)]))
        s.append(input)
    emmission_matrix = pd.DataFrame({'weather': s, 'yes': yes,'no': no})
    emmission_matrix.set_index('weather',inplace=True)
    return emmission_matrix.round(2)

In [8]:
df=loadData()
TransitionProbs=CreateTransitionProbs(df)
print(TransitionProbs)

{'foggy': {'foggy': 131, 'rainy': 76, 'sunny': 54}, 'rainy': {'sunny': 48, 'rainy': 142, 'foggy': 55}, 'sunny': {'foggy': 74, 'sunny': 392, 'rainy': 27}}
       foggy  rainy  sunny
foggy    131     55     74
rainy     76    142     27
sunny     54     48    392
Transistion Probabilities
       foggy  rainy  sunny
foggy   0.50   0.29   0.21
rainy   0.22   0.58   0.20
sunny   0.15   0.05   0.80


In [9]:
emmission_matrix=CreateEmitionProbs(df)
print('Emission Probabilities')
emmission_matrix

Emission Probabilities


,yes,no
weather,,
foggy,0.30,0.70
rainy,0.81,0.19
sunny,0.09,0.91


In [10]:
def fwd(obs1,emmission_matrix, TransitionProbs):
    p_s = 1
    p_r = 0
    p_f = 0
    probabilities_set = []
    for i in range(0,len(obs1)):
        if obs1[i] == 'no':
            print("Observation Seq: ",i, " and visible state is: ",obs1[i])
            print("Prevous Probs: ", p_s,p_r,p_f)
            X_F=((p_f*TransitionProbs.loc['foggy','foggy'])
               +(p_r*TransitionProbs.loc['rainy','foggy'])
               +(p_s*TransitionProbs.loc['sunny','foggy']))* emmission_matrix.loc['foggy','no']
            X_R=((p_f*TransitionProbs.loc['foggy','rainy'])
               +(p_r*TransitionProbs.loc['rainy','rainy'])
               +(p_s*TransitionProbs.loc['sunny','rainy']))* emmission_matrix.loc['rainy','no']
            X_S=((p_f*TransitionProbs.loc['foggy','sunny'])
               +(p_r*TransitionProbs.loc['rainy','sunny'])
               +(p_s*TransitionProbs.loc['sunny','sunny']))* emmission_matrix.loc['sunny','no']
            probabilities_set.append((X_S.round(5),X_R.round(5),X_F.round(5)))
            p_s,p_r,p_f=X_S.round(5),X_R.round(5),X_F.round(5)
            print("Next Probs: ", p_s,p_r,p_f)
        elif obs1[i] == 'yes':
            print("Observation Seq: ",i, " and visible state is: ",obs1[i])
            print("Prevous Probs: ", p_s,p_r,p_f)
            X_F=((p_f*TransitionProbs.loc['foggy','foggy'])
               +(p_r*TransitionProbs.loc['rainy','foggy'])
               +(p_s*TransitionProbs.loc['sunny','foggy']))* emmission_matrix.loc['foggy','yes']
            X_R=((p_f*TransitionProbs.loc['foggy','rainy'])
               +(p_r*TransitionProbs.loc['rainy','rainy'])
               +(p_s*TransitionProbs.loc['sunny','rainy']))* emmission_matrix.loc['rainy','yes']
            X_S=((p_f*TransitionProbs.loc['foggy','sunny'])
               +(p_r*TransitionProbs.loc['rainy','sunny'])
               +(p_s*TransitionProbs.loc['sunny','sunny']))* emmission_matrix.loc['sunny','yes']
            probabilities_set.append((X_S.round(5),X_R.round(5),X_F.round(5)))
            p_s,p_r,p_f=X_S.round(5),X_R.round(5),X_F.round(5)
            print("Next Probs: ", p_s,p_r,p_f)
    return probabilities_set

    
obs1 = ['no', 'no', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'yes']

print('Forward and Viterbi Algorithm Outputs -')
#print(emmission_matrix)
#print(TransitionProbs)
prob_set=fwd(obs1,emmission_matrix,TransitionProbs)
last_sunny,last_rainy,last_foggy=prob_set[-1]

prob_set_arr=np.asarray(prob_set)
prob_set_df = pd.DataFrame({'sunny': prob_set_arr[:, 0], 'rainy': prob_set_arr[:, 1], 'foggy': prob_set_arr[:,2]})
print(prob_set_df)
print(prob_set_df.idxmax(axis=1))

sunny,rainy,foggy=df['input'].value_counts()

print("Final W0 = ",last_foggy*(foggy/1000) + last_rainy*(rainy/1000) + last_sunny*(sunny/1000))


Forward and Viterbi Algorithm Outputs -
Observation Seq:  0  and visible state is:  no
Prevous Probs:  1 0 0
Next Probs:  0.728 0.0095 0.105
Observation Seq:  1  and visible state is:  no
Prevous Probs:  0.728 0.0095 0.105
Next Probs:  0.55178 0.01375 0.11465
Observation Seq:  2  and visible state is:  no
Prevous Probs:  0.55178 0.01375 0.11465
Next Probs:  0.42611 0.01307 0.10018
Observation Seq:  3  and visible state is:  yes
Prevous Probs:  0.42611 0.01307 0.10018
Next Probs:  0.03281 0.04693 0.03506
Observation Seq:  4  and visible state is:  no
Prevous Probs:  0.03281 0.04693 0.03506
Next Probs:  0.03913 0.00742 0.02294
Observation Seq:  5  and visible state is:  no
Prevous Probs:  0.03913 0.00742 0.02294
Next Probs:  0.03422 0.00245 0.01328
Observation Seq:  6  and visible state is:  yes
Prevous Probs:  0.03422 0.00245 0.01328
Next Probs:  0.00276 0.00566 0.00369
Observation Seq:  7  and visible state is:  yes
Prevous Probs:  0.00276 0.00566 0.00369
Next Probs:  0.00037 0.00364 0